In [ ]:
import pandas as pd
import gc

#### Prepare state <=> numeric mapping

In [ ]:
df = pd.read_csv('/opt/data/secc/secc_state_name_age_sex_social_cat.csv.gz', usecols=['state'])
df

In [ ]:
sdf = df.groupby(['state']).agg({'state': 'count'})

In [ ]:
sdf.rename(columns={'state': 'count'}, inplace=True)
sdf.reset_index(inplace=True)

In [ ]:
cat = sdf.state.astype('category').cat

In [ ]:
states_map = dict(zip(cat.categories, cat.codes))

In [ ]:
pd.DataFrame.from_dict(states_map, orient='index', columns=['code']).to_csv('secc_state.csv', index_label='state')

In [ ]:
d = pd.read_csv('secc_state.csv').to_dict(orient='list')
states_map = dict(zip(d['state'], d['code']))
states_map

In [ ]:
social_cats_map = {'st': 't', 'sc': 'c', 'other': 'o'}

In [ ]:
adf = None
count = 0
for df in pd.read_csv('/opt/data/secc/secc_state_name_age_sex_social_cat.csv.gz', usecols=['state', 'name', 'social_cat', 'age', 'gender'], dtype=str, chunksize=1000000):
    count += 1
    print(count)
    df.dropna(subset=['name'], inplace=True)
    df.dropna(subset=['age'], inplace=True)
    df.drop(df[~df.age.str.contains('^[0-9\.]+$')].index, inplace=True)
    df['birth_year'] = 2011 - df.age.astype(float)
    df.birth_year = df.birth_year.astype(int)
    df.state = df.state.apply(lambda c: states_map[c])
    df.social_cat = df.social_cat.apply(lambda c: social_cats_map[c])
    df.name = df.name.str.replace('_', ' ')
    df.name = df.name.str.strip()
    df.drop(df[df.name==' '].index, inplace=True)
    df.drop(df[df.name==''].index, inplace=True)
    df['name_wc'] = df.name.apply(lambda c: len(c.split()))
    df['last_name'] = None
    df.loc[df.name_wc >= 2, 'last_name'] = df.name.apply(lambda c: c.split()[-1])
    df.loc[df.last_name.notnull() & (df.last_name.str.len() <= 2), 'last_name'] = None
    del df['name']
    del df['name_wc']
    del df['age']
    if adf is None:
        adf = df
    else:
        adf = pd.concat([adf, df])
    del df
    gc.collect()
    #if count > 15:
    #    break

In [ ]:
adf.to_csv('/opt/data/secc/secc_state_name_age_sex_social_cat_recode.csv.gz', index=False)